### Евгений Шенк

## Лабораторная раота №5. Спрогнозировать отток клиентов банка по историческим данным на языке Scala

In [1]:
import sys.process._
import scala.collection.mutable.ListBuffer
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.functions.lit
import org.apache.spark.sql.functions.isnull
import org.apache.spark.sql.functions.slice
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.functions.split
import org.apache.spark.sql.functions.expr
import org.apache.spark.ml.linalg.DenseVector
import org.apache.spark.sql.types._
import org.apache.spark.ml.tuning.CrossValidator
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.classification.GBTClassifier
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

val conf = new SparkConf()

val spark = SparkSession
     .builder()
     .config(conf)
     .appName("ESShenk_spark_session")
     .getOrCreate()

In [2]:
"hdfs dfs -ls /labs/slaba05/"!

Found 2 items
-rw-r--r--   3 hdfs hdfs   24271415 2022-01-06 18:46 /labs/slaba05/lab05_test.csv
-rw-r--r--   3 hdfs hdfs  176160768 2022-01-06 18:46 /labs/slaba05/lab05_train.csv


0

In [3]:
// val df_test = spark.read.csv("/labs/slaba05/lab05_test.csv", inferSchema=true, header=true)
// val df_train = spark.read.csv("/labs/slaba05/lab05_train.csv", inferSchema=true, header=true)
val df_test = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/labs/slaba05/lab05_test.csv")
val df_train = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/labs/slaba05/lab05_train.csv")

df_test = [_c0: int, ID: int ... 114 more fields]
df_train = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [4]:
df_train.show(1, 150, true)

-RECORD 0--------------------------------------------
 _c0                         | 333149                
 ID                          | 479990                
 CR_PROD_CNT_IL              | 0                     
 AMOUNT_RUB_CLO_PRC          | 0.0                   
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                   
 APP_REGISTR_RGN_CODE        | null                  
 PRC_ACCEPTS_A_POS           | 0.0                   
 PRC_ACCEPTS_A_TK            | 0.0                   
 TURNOVER_DYNAMIC_IL_1M      | 0.0                   
 CNT_TRAN_AUT_TENDENCY1M     | null                  
 SUM_TRAN_AUT_TENDENCY1M     | null                  
 AMOUNT_RUB_SUP_PRC          | 0.0                   
 PRC_ACCEPTS_A_AMOBILE       | 0.0                   
 SUM_TRAN_AUT_TENDENCY3M     | null                  
 CLNT_TRUST_RELATION         | null                  
 PRC_ACCEPTS_TK              | 0.0                   
 PRC_ACCEPTS_A_MTP           | 0.0                   
 REST_DYNAMIC_FDEP_1M       

In [5]:
var NullfilteredCols = new ListBuffer[String]();
for (column <- df_train.columns){
    val nulls = df_train.select(column).filter(isnull(col(column))).count();
    println(column + ": " + nulls);
    if (nulls < 200000){
        if (column != "TARGET"){
            NullfilteredCols += column;
        }
    }
}

_c0: 0
ID: 0
CR_PROD_CNT_IL: 0
AMOUNT_RUB_CLO_PRC: 34550
PRC_ACCEPTS_A_EMAIL_LINK: 180814
APP_REGISTR_RGN_CODE: 265914
PRC_ACCEPTS_A_POS: 180814
PRC_ACCEPTS_A_TK: 180814
TURNOVER_DYNAMIC_IL_1M: 1
CNT_TRAN_AUT_TENDENCY1M: 251229
SUM_TRAN_AUT_TENDENCY1M: 251229
AMOUNT_RUB_SUP_PRC: 34551
PRC_ACCEPTS_A_AMOBILE: 180814
SUM_TRAN_AUT_TENDENCY3M: 220560
CLNT_TRUST_RELATION: 257829
PRC_ACCEPTS_TK: 180814
PRC_ACCEPTS_A_MTP: 180814
REST_DYNAMIC_FDEP_1M: 1
CNT_TRAN_AUT_TENDENCY3M: 220560
CNT_ACCEPTS_TK: 180814
APP_MARITAL_STATUS: 258989
REST_DYNAMIC_SAVE_3M: 1
CR_PROD_CNT_VCU: 1
REST_AVG_CUR: 1
CNT_TRAN_MED_TENDENCY1M: 258307
APP_KIND_OF_PROP_HABITATION: 267042
CLNT_JOB_POSITION_TYPE: 280242
AMOUNT_RUB_NAS_PRC: 34551
CLNT_JOB_POSITION: 130371
APP_DRIVING_LICENSE: 268868
TRANS_COUNT_SUP_PRC: 34551
APP_EDUCATION: 259099
CNT_TRAN_CLO_TENDENCY1M: 260941
SUM_TRAN_MED_TENDENCY1M: 258307
PRC_ACCEPTS_A_ATM: 180814
PRC_ACCEPTS_MTP: 180814
TRANS_COUNT_NAS_PRC: 34551
APP_TRAVEL_PASS: 268868
CNT_ACCEPTS_MTP: 

NullfilteredCols = ListBuffer(_c0, ID, CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_ACCEPTS_TK, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, AMOUNT_RUB_NAS_PRC, CLNT_JOB_POSITION, TRANS_COUNT_SUP_PRC, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, CNT_ACCEPTS_MTP, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, AGE, CR_PROD_CNT_CC, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_1M, CR_PROD_CNT_CCFP, REST_DYNAMIC_CUR_1M, REST_AVG_PAYM, LDEAL_GRACE_DAYS_PCT_MED, REST_DYNAMIC_CUR_3M, CNT_TRAN_SUP_TENDEN...


ListBuffer(_c0, ID, CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_ACCEPTS_TK, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, AMOUNT_RUB_NAS_PRC, CLNT_JOB_POSITION, TRANS_COUNT_SUP_PRC, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, CNT_ACCEPTS_MTP, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, AGE, CR_PROD_CNT_CC, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_1M, CR_PROD_CNT_CCFP, REST_DYNAMIC_CUR_1M, REST_AVG_PAYM, LDEAL_GRACE_DAYS_PCT_MED, REST_DYNAMIC_CUR_3M, CNT_TRAN_SUP_TENDEN...

In [6]:
val NullfilteredColsList = NullfilteredCols.toList
NullfilteredCols += "TARGET"
val NullfilteredColsTargetList = NullfilteredCols.toList

NullfilteredColsList = List(_c0, ID, CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_ACCEPTS_TK, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, AMOUNT_RUB_NAS_PRC, CLNT_JOB_POSITION, TRANS_COUNT_SUP_PRC, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, CNT_ACCEPTS_MTP, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, AGE, CR_PROD_CNT_CC, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_1M, CR_PROD_CNT_CCFP, REST_DYNAMIC_CUR_1M, REST_AVG_PAYM, LDEAL_GRACE_DAYS_PCT_MED, REST_DYNAMIC_CUR_3M, CNT_TRAN_SUP_TENDENCY3M, TURNOVER_DYNAMIC_CUR_1M, RE...


List(_c0, ID, CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_ACCEPTS_TK, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, AMOUNT_RUB_NAS_PRC, CLNT_JOB_POSITION, TRANS_COUNT_SUP_PRC, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, CNT_ACCEPTS_MTP, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, AGE, CR_PROD_CNT_CC, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_1M, CR_PROD_CNT_CCFP, REST_DYNAMIC_CUR_1M, REST_AVG_PAYM, LDEAL_GRACE_DAYS_PCT_MED, REST_DYNAMIC_CUR_3M, CNT_TRAN_SUP_TENDENCY3M, TURNOVER_DYNAMIC_CUR_1M, RE...

In [7]:
val df_train_pure = df_train.select(NullfilteredColsTargetList.map(col): _*).filter(col("age").isNotNull).na.fill(0)
val df_test_pure = df_test.select(NullfilteredColsList.map(col): _*).na.fill(0)

df_train_pure = [_c0: int, ID: int ... 62 more fields]
df_test_pure = [_c0: int, ID: int ... 61 more fields]


[_c0: int, ID: int ... 61 more fields]

In [8]:
df_train_pure.show(5, 150, true)

-RECORD 0---------------------------------------
 _c0                      | 333149              
 ID                       | 479990              
 CR_PROD_CNT_IL           | 0                   
 AMOUNT_RUB_CLO_PRC       | 0.0                 
 PRC_ACCEPTS_A_EMAIL_LINK | 0.0                 
 PRC_ACCEPTS_A_POS        | 0.0                 
 PRC_ACCEPTS_A_TK         | 0.0                 
 TURNOVER_DYNAMIC_IL_1M   | 0.0                 
 AMOUNT_RUB_SUP_PRC       | 0.0                 
 PRC_ACCEPTS_A_AMOBILE    | 0.0                 
 PRC_ACCEPTS_TK           | 0.0                 
 PRC_ACCEPTS_A_MTP        | 0.0                 
 REST_DYNAMIC_FDEP_1M     | 0.0                 
 CNT_ACCEPTS_TK           | 0.0                 
 REST_DYNAMIC_SAVE_3M     | 0.0                 
 CR_PROD_CNT_VCU          | 0                   
 REST_AVG_CUR             | 19069.6917166667    
 AMOUNT_RUB_NAS_PRC       | 0.0                 
 CLNT_JOB_POSITION        | null                
 TRANS_COUNT_SUP_PRC

### Model

In [9]:
val ignore_cols = List[String]("ID", "TARGET", "CLNT_JOB_POSITION", "PACK", "_c0")
val unwanted = ignore_cols.toSet
val toModelCols = NullfilteredColsList.filterNot(unwanted)
val assembler = new VectorAssembler().setInputCols(toModelCols.toArray).setOutputCol("features")

ignore_cols = List(ID, TARGET, CLNT_JOB_POSITION, PACK, _c0)
unwanted = Set(ID, _c0, PACK, TARGET, CLNT_JOB_POSITION)
toModelCols = List(CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_ACCEPTS_TK, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, AMOUNT_RUB_NAS_PRC, TRANS_COUNT_SUP_PRC, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, CNT_ACCEPTS_MTP, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, AGE, CR_PROD_CNT_CC, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_1M, CR_PROD_CNT_CCFP, ...


List(CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_ACCEPTS_TK, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, AMOUNT_RUB_NAS_PRC, TRANS_COUNT_SUP_PRC, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, CNT_ACCEPTS_MTP, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, AGE, CR_PROD_CNT_CC, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_IL_1M, CR_PROD_CNT_CCFP, ...

In [10]:
val train_data = assembler.transform(df_train_pure).select("ID", "TARGET", "features").repartition(16).cache()
val test_data = assembler.transform(df_test_pure).select("ID", "features").repartition(16).cache()
train_data.count()
test_data.count()

train_data = [ID: int, TARGET: int ... 1 more field]
test_data = [ID: int, features: vector]


44399

In [11]:
val gbt = new GBTClassifier().setMaxIter(10).setMaxDepth(4).setSeed(27).setLabelCol("TARGET")
val evaluator = new BinaryClassificationEvaluator().setRawPredictionCol("probability").setLabelCol("TARGET").setMetricName("areaUnderROC")
val paramGrid = new ParamGridBuilder().build()

gbt = gbtc_904fb1c6ae2c
evaluator = binEval_c76dcaea6a2c
paramGrid = 


Array({
})


In [12]:
val crossval = new CrossValidator().setEstimator(gbt).setEstimatorParamMaps(paramGrid).setEvaluator(evaluator).setNumFolds(5).setParallelism(1)

crossval = cv_ccf0a79ba022


cv_ccf0a79ba022

In [13]:
val model = crossval.fit(train_data)

model = cv_ccf0a79ba022


cv_ccf0a79ba022

In [14]:
val auc_roc = model.avgMetrics
auc_roc

auc_roc = Array(0.803962634248532)


Array(0.803962634248532)

In [15]:
val preds = model.transform(test_data.select("ID", "features"))

preds = [ID: int, features: vector ... 3 more fields]


[ID: int, features: vector ... 3 more fields]

In [16]:
val vectorToColumn = udf{ (x:DenseVector, index: Int) => x(index) }

vectorToColumn = UserDefinedFunction(<function2>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7, IntegerType)))


UserDefinedFunction(<function2>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7, IntegerType)))

In [17]:
val preds_to_save = preds.select(col("ID").alias("id"), vectorToColumn(col("probability"), lit(1)).alias("target"))

preds_to_save = [id: int, target: double]


[id: int, target: double]

In [18]:
preds_to_save.show()

+------+-------------------+
|    id|             target|
+------+-------------------+
|236860|0.15128360712441302|
|164580|0.12139759588697352|
|464747|0.15375332696253308|
|389522|0.07125990768291113|
|174610|0.18616327163597723|
|582858|0.23594852021716295|
|360126|0.07194197937079927|
|535823|0.10321548276707526|
|309363|0.11328393845015106|
|327676|0.07125990768291113|
|189201| 0.1944248491555688|
|239273|0.13217943311623503|
|148621| 0.1561183049323277|
|357299|0.07125990768291113|
|195814|0.07770775350867676|
|404823|0.07340333620996564|
|436574|0.07125990768291113|
|392436| 0.3292734086313257|
|511015|0.10756973461466812|
|162798|  0.334462340545607|
+------+-------------------+
only showing top 20 rows



### Сохранить и выйти

In [19]:
preds_to_save.coalesce(1).write
    .option("header", "true")
    .option("delimiter","\t")
    .mode("overwrite")
    .csv("lab05.csv")

In [20]:
spark.stop()